In [1]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# from sklearn.metrics import log_loss, f1_score, fbeta_score, recall_score, precision_score, confusion_matrix
import urllib.request, json
from skimage.filters import threshold_otsu
from pprint import pprint

# Homemade functions required
from data_prep_functions import *
from interpro_scraping import interpro_scraping_pandas
from uniprot_netsurfp_scraping import *

# Generate features and names from predownloaded properties and LC-MS/MS data 

In [2]:
############# EDIT, FILL IN DATA HERE ############
data_file = '2020-02-03 MS compiled for ML project.xlsx'  # needs xlsx at the end 

plasma_prop_sheet = 'Protein properties, plasma'
plasma_mass_spec_sheet = 'PNPs, plasma' #'(GT)6-SWCNT, plasma'

csf_prop_sheet = 'Protein properties, CSF'
csf_mass_spec_sheet = 'PNPs, CSF' #'(GT)15-SWCNT, CSF'
###################################################

In [3]:
############ NO EDITS REQUIRED ##############

data_filepath = "data/" + data_file

# load data

plasma_raw_data = pd.read_excel(data_filepath, sheet_name= plasma_prop_sheet, thousands=',')
csf_raw_data = pd.read_excel(data_filepath, sheet_name= csf_prop_sheet, thousands=',')

plasma_mass_spec_data = pd.read_excel(data_filepath, sheet_name=plasma_mass_spec_sheet, header=2, thousands=',')
csf_mass_spec_data = pd.read_excel(data_filepath, sheet_name=csf_mass_spec_sheet, header=2, thousands=',')

# clean up data 

plasma_cleaned_data = clean_up_data_biopy(plasma_raw_data)# clean_up_data_v2(plasma_raw_data) 
csf_cleaned_data = clean_up_data_biopy(csf_raw_data) #clean_up_data_v2(csf_raw_data) 


plasma_cleaned_data, csf_cleaned_data = normalize_mass_length(plasma_cleaned_data, csf_cleaned_data)

/home/randhal/.local/lib/python3.14/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
/home/randhal/.local/lib/python3.14/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(


In [4]:
netsurfp_data = pd.read_excel("data/"+'netsurfp_2_lcms_proteins_processed_updated.xlsx')

plasma_complete_data = pd.merge(plasma_cleaned_data, netsurfp_data, left_on='Entry', right_on='entry')
csf_complete_data = pd.merge(csf_cleaned_data, netsurfp_data, left_on='Entry', right_on='entry')

for df in [plasma_complete_data, csf_complete_data]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

csf_complete_data.head()

,Entry,Protein names,Sequence,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,...,fraction_exposed_exposed_S,fraction_exposed_exposed_T,fraction_exposed_exposed_V,fraction_exposed_exposed_W,fraction_exposed_exposed_Y,nsp_secondary_structure_coil,nsp_secondary_structure_sheet,nsp_secondary_structure_helix,nsp_disordered,asa_sum_normalized
0,P02768,Serum albumin,MKWVTFISLLFLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKAL...,609,69367,0.103448,0.057471,0.059113,0.101806,0.057471,...,0.050000,0.067647,0.035294,0.002941,0.011765,0.286,0.000,0.714,0.038,0.464794
1,P02766,Transthyretin (ATTR) (Prealbumin) (TBPA),MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,147,15887,0.102041,0.013605,0.034014,0.088435,0.040816,...,0.105263,0.063158,0.042105,0.010526,0.021053,0.551,0.381,0.068,0.197,0.602589
2,P01009,Alpha-1-antitrypsin (Alpha-1 protease inhibito...,MPSSVSWGILLLAGLCCLVPVSLAEDPQGDAAQKTDTSHHDQDHPT...,418,46737,0.062201,0.007177,0.057416,0.076555,0.064593,...,0.076577,0.067568,0.031532,0.004505,0.000000,0.400,0.330,0.270,0.105,0.463058
3,P41222,Prostaglandin-H2 D-isomerase (EC 5.3.99.2) (Be...,MATHHTLWMGLALLGVLGDLQAAPEAQVSVQPNFQQDKFLGRWFSA...,190,21029,0.089474,0.021053,0.042105,0.047368,0.047368,...,0.071429,0.095238,0.039683,0.023810,0.000000,0.463,0.400,0.137,0.137,0.568527
4,P02787,Serotransferrin (Transferrin) (Beta-1 metal-bi...,MRLAVGALLVCAVLGLCLAVPDKTVRWCAVSEHEATKCQSFRDHMK...,698,77064,0.087393,0.057307,0.064470,0.060172,0.040115,...,0.062130,0.047337,0.026627,0.000000,0.005917,0.474,0.186,0.340,0.030,0.419599


In [5]:
###### MASS SPEC DATA, LIKELY REQUIRED (THRESHOLD METHOD) ########

plasma_labels = plasma_mass_spec_data[['Accession', 'NP average','NP fold change']]
csf_labels = csf_mass_spec_data[['Accession', 'NP average', 'NP fold change']]

## For OTSU, Comment out if not using ###

# plasma_labels['Corona'] = (plasma_labels['NP average'] > criteria_plas).astype(int)
# csf_labels['Corona'] = (csf_labels['NP average'] > criteria_csf).astype(int)

## for TRAVIS, Comment out if not using ## 

thresh_power = 2.25 # change based on previous cell
fluids_type_list=['plasma', 'csf']
data_sheets = [plasma_labels, csf_labels]
for k in range(2):
    sheet = data_sheets[k]
    fluid_type = fluids_type_list[k]

    print(f'\n{fluid_type.capitalize()} running')

    num_proteins = np.zeros(100)
    for i in np.arange(0,100,1):
        thresh = i/100
        index = (sheet['NP average']>thresh)
        num_proteins[i] = (np.count_nonzero(index))

    x = np.arange(0,1,.01)
    num_proteins_above_1_abundance = min(num_proteins)
    biexponent_dist = num_proteins - num_proteins_above_1_abundance
    num_protein_thresh = np.max(biexponent_dist)/pow(np.e,thresh_power)
    cutoff_thresh_value = min(x[(biexponent_dist<num_protein_thresh)&(biexponent_dist>0)], default=0)
    sheet['Corona'] = ((sheet['NP average']>cutoff_thresh_value) | (sheet['NP fold change']>1)).astype(int)
    


### end different thresh methods



plasma_labels = plasma_labels.drop(['NP average', 'NP fold change'], axis=1)
csf_labels = csf_labels.drop(['NP average', 'NP fold change'], axis=1)



# checks for any accession lists
plasma_labels = accession_expansion(plasma_labels) 
csf_labels = accession_expansion(csf_labels)

plasma_total_data_names = pd.merge(plasma_labels, plasma_complete_data, left_on='Accession', right_on='Entry')
csf_total_data_names = pd.merge(csf_labels, csf_complete_data, left_on='Accession', right_on='Entry')

plasma_to_export =plasma_total_data_names.drop(['Accession','Entry', 'entry', 'Sequence', 'Length', 'Mass'], axis=1)
csf_to_export = csf_total_data_names.drop(['Accession','Entry', 'entry', 'Sequence', 'Length', 'Mass'], axis=1)

plasma_to_export.to_excel("data/"+'pnp_plasma_features_names_biopy_gravy.xlsx')
csf_to_export.to_excel("data/"+'pnp_csf_features_names_biopy_gravy.xlsx')



Plasma running

Csf running


/tmp/ipykernel_53607/4259391202.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet['Corona'] = ((sheet['NP average']>cutoff_thresh_value) | (sheet['NP fold change']>1)).astype(int)
/tmp/ipykernel_53607/4259391202.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet['Corona'] = ((sheet['NP average']>cutoff_thresh_value) | (sheet['NP fold change']>1)).astype(int)


# Download protein property data

 ## use this if you have a list of protein targets

In [6]:
# only_uniprot_data = pd.read_excel('uniprot_1000_cytoplasm.xlsx')['Entry'] # for use when creating no netsurfp sets
chosen_list = ["Q01995", 'Q13148', 'P16070', 'P00698', 'P00338', 'P07998', 'Q8MU52', 'O00560'] 
test_data = uniprot_data_scraping(chosen_list)  # replace with only_uniprot_data for no netsurf p

Attempting to fetch 8 entries in a single API call.
Successfully retrieved data for 7 out of 8 requested accessions.


In [7]:
test_data

,Entry,Protein names,Reviewed,Protein families,Length,Mass,Sequence,Function [CC],Catalytic activity,EC number,Pathway,Interacts with,Subunit structure,Gene Ontology (biological process),Subcellular location [CC],Chain,Disulfide bond,Glycosylation,Domain [CC],Repeat
0,O00560,Syntenin-1 (Melanoma differentiation-associate...,reviewed,0,298,32444,MSLYPSLEDLKVDKVIQAQTAFSANPANPAILSEASAPIPHDGNLY...,FUNCTION: Multifunctional adapter protein invo...,0,0,0,Q9NYB9; Q9NX04; Q7Z3H0-1; Q8N2N9-4; Q6AI12; Q9...,SUBUNIT: Monomer and homodimer (By similarity)...,actin cytoskeleton organization [GO:0030036]; ...,"SUBCELLULAR LOCATION: Cell junction, focal adh...","CHAIN 2..298; /note=""Syntenin-1""; /id=""PRO_000...",0,0,0,0
1,P00338,L-lactate dehydrogenase A chain (LDH-A) (EC 1....,reviewed,"LDH/MDH superfamily, LDH family",332,36689,MATLKDQLIYNLLKEEQTPQNKITVVGVGAVGMACAISILMKDLAD...,FUNCTION: Interconverts simultaneously and ste...,CATALYTIC ACTIVITY: Reaction=(S)-lactate + NAD...,1.1.1.27,PATHWAY: Fermentation; pyruvate fermentation t...,P11142; P00338; P07195; P07864; P07195; A0A286...,SUBUNIT: Homotetramer (PubMed:11276087). Inter...,glucose catabolic process to lactate via pyruv...,SUBCELLULAR LOCATION: Cytoplasm.,"CHAIN 2..332; /note=""L-lactate dehydrogenase A...",0,0,0,0
2,P07998,Ribonuclease pancreatic (EC 4.6.1.18) (HP-RNas...,reviewed,Pancreatic ribonuclease family,156,17644,MALEKSLVRLLLLVLILLVLGWVQPSLGKESRAKKFQRQHMDSDSS...,FUNCTION: Endonuclease that catalyzes the clea...,CATALYTIC ACTIVITY: Reaction=an [RNA] containi...,4.6.1.18,0,P13489; Q9C029,SUBUNIT: Monomer. Interacts with and forms tig...,defense response to Gram-positive bacterium [G...,SUBCELLULAR LOCATION: Secreted.,"CHAIN 29..156; /note=""Ribonuclease pancreatic""...",DISULFID 54..112; DISULFID 68..123; DISULFID 8...,"CARBOHYD 62; /note=""N-linked (GlcNAc...) aspar...",0,0
3,P00698,"Lysozyme C (EC 3.2.1.17) (1,4-beta-N-acetylmur...",reviewed,Glycosyl hydrolase 22 family,147,16239,MRSLLILVLCFLPLAALGKVFGRCELAAAMKRHGLDNYRGYSLGNW...,FUNCTION: Lysozymes have primarily a bacteriol...,CATALYTIC ACTIVITY: Reaction=Hydrolysis of (1-...,3.2.1.17,0,P00698; P0C0V0,SUBUNIT: Monomer.,cell wall macromolecule catabolic process [GO:...,SUBCELLULAR LOCATION: Secreted.,"CHAIN 19..147; /note=""Lysozyme C""; /id=""PRO_00...",DISULFID 24..145; DISULFID 48..133; DISULFID 8...,0,0,0
4,Q13148,TAR DNA-binding protein 43 (TDP-43),reviewed,0,414,44740,MSEYIRVTEDENDEPIEIPSEDDGTVLLSTVTAQFPGACGLRYRNP...,FUNCTION: RNA-binding protein that is involved...,0,0,0,Q969K4; P55265; Q6ZTN6-2; P13928; P63010-2; P0...,"SUBUNIT: Homodimer (PubMed:20043239, PubMed:24...",3'-UTR-mediated mRNA destabilization [GO:00611...,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,"CHAIN 1..414; /note=""TAR DNA-binding protein 4...",0,0,DOMAIN: Consists of an N-terminal domain (NTD)...,0
5,P16070,CD44 antigen (CDw44) (Epican) (Extracellular m...,reviewed,0,742,81538,MDKFWWHAAWGLCLVPLSLAQIDLNITCRFAGVFHVEKNGRYSISR...,FUNCTION: Cell-surface receptor that plays a r...,0,0,0,P04233; P26038; Q9UPY5; P18011,SUBUNIT: Interacts with PKN2 (PubMed:15123640)...,cartilage development [GO:0051216]; cell adhes...,SUBCELLULAR LOCATION: Cell membrane {ECO:00002...,"CHAIN 21..742; /note=""CD44 antigen""; /id=""PRO_...","DISULFID 28..129; /evidence=""ECO:0000269|PubMe...","CARBOHYD 25; /note=""N-linked (GlcNAc...) aspar...",DOMAIN: The lectin-like LINK domain is respons...,0
6,Q01995,Transgelin (22 kDa actin-binding protein) (Pro...,reviewed,Calponin family,201,22611,MANKGPSYGMSREVQSKIEKKYDEELEERLVEWIIVQCGPDVGRPD...,FUNCTION: Actin cross-linking/gelling protein ...,0,0,0,Q9H8Y8; P08069; Q9BSI4,0,epithelial cell differentiation [GO:0030855]; ...,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000305}.,"CHAIN 2..201; /note=""Transgelin""; /id=""PRO_000...",0,0,0,"REPEAT 175..200; /note=""Calponin-like"""


In [8]:

test_data['Calcium binding'] = 0
test_data['Activity regulation'] = 0
test_data.head()

for i in range(8):

    testing = test_data.iloc[i*1:i*1 + 1]
    try:
        testing_cleaned = clean_up_data_biopy(testing)
    except:
        print(f'error at {i}')

test_data_corrected = test_data.copy()#.drop([405, 680, 707, 792]) # run above once and run again with error drops if there are any
test_cleaned_data = clean_up_data_biopy(test_data_corrected)


plasma_raw_data = pd.read_excel(data_filepath, sheet_name= plasma_prop_sheet, thousands=',')
plasma_cleaned_data = clean_up_data_biopy(plasma_raw_data)

test_cleaned_data['length'] = test_cleaned_data['Length'] / plasma_cleaned_data.Length.max()
test_cleaned_data['mass'] = test_cleaned_data['Mass'] / plasma_cleaned_data.Mass.max()
test_cleaned_data['molecular_weight'] = test_cleaned_data['molecular_weight'] / plasma_cleaned_data.molecular_weight.max()
plasma_cleaned_data, test_cleaned_data = normalize_mass_length(plasma_cleaned_data, test_cleaned_data)

error at 7


/home/randhal/proj/Carbon_Nanotube/swnt-protein-corona-ML/data_prep_functions.py:338: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_data = raw_data.fillna(0) # fill nans
/home/randhal/.local/lib/python3.14/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
/home/randhal/proj/Carbon_Nanotube/swnt-protein-corona-ML/data_prep_functions.py:338: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_opti

In [9]:
netsurfp_test_data = pd.read_excel("data/"+'netsurfp_2_proteins_selected_for_testing_processed_updated.xlsx')  # load in the netsurfp data from the same set of data

test_complete_data = pd.merge(test_cleaned_data, netsurfp_test_data, left_on='Entry', right_on='entry')


for df in [test_complete_data]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

test_complete_data.head()

,Entry,Protein names,Sequence,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,...,fraction_exposed_exposed_S,fraction_exposed_exposed_T,fraction_exposed_exposed_V,fraction_exposed_exposed_W,fraction_exposed_exposed_Y,nsp_secondary_structure_coil,nsp_secondary_structure_sheet,nsp_secondary_structure_helix,nsp_disordered,asa_sum_normalized
0,O00560,Syntenin-1 (Melanoma differentiation-associate...,MSLYPSLEDLKVDKVIQAQTAFSANPANPAILSEASAPIPHDGNLY...,298,32444,0.077181,0.010067,0.050336,0.050336,0.030201,...,0.092683,0.058537,0.034146,0.000000,0.014634,0.517,0.248,0.235,0.138,0.540409
1,P00338,L-lactate dehydrogenase A chain (LDH-A) (EC 1....,MATLKDQLIYNLLKEEQTPQNKITVVGVGAVGMACAISILMKDLAD...,332,36689,0.054217,0.015060,0.054217,0.054217,0.021084,...,0.036585,0.054878,0.024390,0.012195,0.042683,0.319,0.205,0.476,0.006,0.433721
2,P07998,Ribonuclease pancreatic (EC 4.6.1.18) (HP-RNas...,MALEKSLVRLLLLVLILLVLGWVQPSLGKESRAKKFQRQHMDSDSS...,156,17644,0.032051,0.051282,0.038462,0.044872,0.025641,...,0.147826,0.052174,0.060870,0.008696,0.026087,0.622,0.199,0.179,0.205,0.610436
3,P00698,"Lysozyme C (EC 3.2.1.17) (1,4-beta-N-acetylmur...",MRSLLILVLCFLPLAALGKVFGRCELAAAMKRHGLDNYRGYSLGNW...,147,16239,0.095238,0.061224,0.047619,0.013605,0.027211,...,0.087912,0.076923,0.032967,0.000000,0.021978,0.537,0.075,0.388,0.122,0.550834
4,Q13148,TAR DNA-binding protein 43 (TDP-43),MSEYIRVTEDENDEPIEIPSEDDGTVLLSTVTAQFPGACGLRYRNP...,414,44740,0.062802,0.014493,0.053140,0.053140,0.053140,...,0.126246,0.019934,0.016611,0.016611,0.009967,0.676,0.198,0.126,0.341,0.576662


In [10]:
# test_data_done=test_complete_data.drop(['Entry', 'entry', 'Sequence', 'Length', 'Mass', 'Accession', 'Unnamed: 0'], axis=1)
test_data_done=test_complete_data.drop(['Entry', 'entry', 'Sequence', 'Unnamed: 0', 'Length', 'Mass'], axis=1)
# test_data_done.head()
test_data_done.shape
test_data_done.to_excel("data/"+'proteins_selected_for_testing_complete_updated.xlsx')

## use this if you want to download data based off of a link

In [11]:
### done to check bias in our classifier #### 
covid_link = 'https://covid-19.uniprot.org/uniprotkb?query=id&format=html'
cytoplasm_link = 'https://www.uniprot.org/uniprot/?query=locations:(location:%22Cytoplasm%20%5BSL-0086%5D%22)&fil=organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22+AND+reviewed%3Ayes&limit=75'
dendritic_spine ='https://www.uniprot.org/uniprot/?query=locations:(location:%22Dendritic%20spine%20%5BSL-0284%5D%22)&fil=organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22+AND+reviewed%3Ayes&limit=150'
clatherin_pit = 'https://www.uniprot.org/uniprot/?query=locations:(location:%22Clathrin-coated%20pit%20%5BSL-0069%5D%22)&fil=organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22+AND+reviewed%3Ayes'
nucleus = 'https://www.uniprot.org/uniprot/?query=locations%3A%28location%3A%22Nucleus+%5BSL-0191%5D%22%29+reviewed%3Ayes+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22&sort=score&limit=120'

entry_check = pd.read_html(nucleus, header=0)
entry_list = entry_check[0]['Entry']

verification_set = uniprot_data_scraping(entry_list)

verification_set['Calcium binding'] = 0
verification_set['Activity regulation'] = 0
verification_cleaned_data = clean_up_data_biopy(verification_set)


plasma_raw_data = pd.read_excel(data_file, sheet_name= plasma_prop_sheet, thousands=',')
plasma_cleaned_data = clean_up_data_biopy(plasma_raw_data)
plasma_cleaned_data, test_cleaned_data = normalize_mass_length(plasma_cleaned_data, verification_cleaned_data)

# netsurfp_verification_data = pd.read_excel('netsurfp_verification_data_processed_nucleus.xlsx')

# verification_complete_data = pd.merge(verification_cleaned_data, netsurfp_verification_data, left_on='Entry', right_on='entry')


# for df in [verification_complete_data]:
#     for col in ['asa_sum']:
#         df[col+'_normalized'] = df[col] / df['Mass']
test_cleaned_data.head()
# verification_complete_data.head()

ValueError: No tables found